In [1]:
# Importing the required libraries (install in your environment first)
import numpy as np
import pandas as pd
from math import log2, sqrt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import plotly.express as px
import plotly.graph_objects as go

## Import Data to Dataframe

In [22]:
data_filePath = 'QRNGvsPRNG_TrainingData.txt'
df = pd.read_csv(data_filePath, sep=' ', header=["data", "label"], dtype={"data": str, "label": np.int64})
df.columns = ["data", "label"]
df.head()

ValueError: header must be integer or list of integers

### Process Labels and Train_Test_split

In [21]:
df['data_length_str'] = df['data'].apply(lambda x: len(str(x)))
df['data_length_str']


0        29
1        30
2        30
3        30
4        31
         ..
23995    29
23996    31
23997    30
23998    29
23999    30
Name: data_length_str, Length: 24000, dtype: int64

In [4]:
df['label'].value_counts()

label
1    12000
2    12000
Name: count, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X = df['data'].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19200, 1), (4800, 1), (19200, 1), (4800, 1))

In [8]:
X_train.min()

40374992047913770101411226

## Train Classifier

In [8]:
from sklearn.linear_model import LogisticRegression
# Create the Gradient Boosting classifier
model = LogisticRegression()

In [9]:
# Train the model. Define X_train and y_train from your training dataframe using 
# sklearns train_test_split method
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy_gb)

Gradient Boosting Accuracy:  0.498125


/Users/yubraj/Library/Caches/pypoetry/virtualenvs/wigglerng-jNs1t9jR-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
